In [1]:
using Fermi
using Fermi.Integrals
using LinearAlgebra
using BenchmarkTools
using Combinatorics
using OMEinsum

## Hartree-Fock Algorithm 
*Using the notation of Szabo & Ostlund - Introduction to Advanced Electronic Structure Theory*


In [2]:
@molecule {
    O   0.000000     0.00000     0.11779
    H   0.000000     0.75545    -0.47116
    H   0.000000    -0.75545    -0.47116 
}
@set {
    basis sto-3g
    charge 0
    multiplicity 1 
}

ao_integrals = IntegralHelper(eri_type=Chonky())

function RHF(ao_integrals)
    
    #1. Calculate all the required molecular integrals (using fermi.jl)
    ao_integrals = IntegralHelper(eri_type=Chonky())
    S = ao_integrals["S"]
    Hcore = Array(ao_integrals["T"]) + Array(ao_integrals["V"])
    V_nuc = ao_integrals.molecule.Vnuc
    μνλσ = Array(ao_integrals["ERI"])
    
    #2. Store basis set size K and number of occupied orbitals nocc
    K = size(S, 1)
    nocc = ao_integrals.molecule.Nα
    
    #3. Diagonalize the overlap matrix S and obtain a transformation matrix (3.167)
    s, U = eigen(S)
    s = Diagonal(s)
    X = U * inv(sqrt(s)) * U'
    
    #4. Obtain a guess at the density matrix Pnew
    C = zeros(K,K)
    occupied = Diagonal(vcat(fill(2, nocc), zeros(K-nocc)))
    Pnew = C * occupied * C'
    
    #5. Calculate matrix G of eq (3.154) (in two steps here)
    G = μνλσ - 0.5 * permutedims(μνλσ, (1,4,3,2))
    
    iteration = 1
    Ptol = 10
    maxiter = 50
    E = 0
    while Ptol > 1e-4
        P = Pnew
        Gμν = ein"λσ, μνλσ -> μν"(P,G)
        
        #6. Add G to the core Hamiltonian to obtain the Fock matrix
        F = Hcore + Gμν
    
        #7. Calculate the transformed Fock matrix Ft
        Ft = X' * F * X
        
        #8. Diagonalize Ft to obtain Ct and ε
        ε, Ct = eigen(Ft, sortby = x->x)
    
        #9. Calculate C
        C = X * Ct
    
        #10. Form a new density matrix P from C using (3.145)
        Pnew = C * occupied * C'
        
        #11. Determine whether the procedure has converged
        Ptol = sqrt.(sum((Pnew .- P).^2) ./ K^2)
    
        E = ein"νμ,μν -> "(Pnew,(Hcore+F))
        E = 0.5 .* E .+ V_nuc
        println("Iteration $iteration ---- E = $E ---- Ptol = $Ptol")
        if iteration > maxiter
            println("Max number of iterations reached")
            break
        end
        iteration += 1
    end
    return E, C
    println("Hartree Fock converged")
end

#Run the Hartee-Fock Calculation
E, C = RHF(ao_integrals)
println("Converged Hartree-Fock Energy = $E")

Iteration 1 ---- E = -118.16877249768544 ---- Ptol = 0.769869784390838
Iteration 2 ---- E = -71.19755398663607 ---- Ptol = 0.5276234409815913
Iteration 3 ---- E = -75.28186975157045 ---- Ptol = 0.053767482192072696
Iteration 4 ---- E = -74.9375074876783 ---- Ptol = 0.01050624624544192
Iteration 5 ---- E = -74.96647042201126 ---- Ptol = 0.004072569282002601
Iteration 6 ---- E = -74.96355897322383 ---- Ptol = 0.0016206824452923913
Iteration 7 ---- E = -74.96341516996763 ---- Ptol = 0.0006797065105307897
Iteration 8 ---- E = -74.96325755180108 ---- Ptol = 0.00028923338599482905
Iteration 9 ---- E = -74.96319518881691 ---- Ptol = 0.00012418820789279986
Iteration 10 ---- E = -74.96316753236361 ---- Ptol = 5.355248914878888e-5
Converged Hartree-Fock Energy = -74.96316753236361


In [120]:
#Check that everything went well with fermi.jl
#wfn = @energy rhf
#wfn.orbitals.C

## Full CI
N-resolution method, code adapted in Julia from pyscf: <https://github.com/pyscf/pyscf/blob/master/pyscf/fci/fci_slow.py> <br><br>
Following the notation of *Molecular Electronic-Structure Theory by Helgaker, Jorgensen, Olsen*

We will use the alpha and beta string representation of the CI expansion

$$ \ket{I_\alpha I_\beta} = \hat{\alpha}_{I_\alpha}\hat{\beta}_{I_\beta} \ket{vac} $$ 

$$ I_{\alpha} = "00000111" = 7 $$

### Backtracking algorithm to generate  all occupation bitstrings

In [6]:
nmo = 7
nα = nβ = 5
nvir = nmo - nα
orbital_list = collect(0:nmo-1)

function generate_string_iter(orbital_list,nelec)
	if nelec == 1
	    result = [(1 << i) for i in orbital_list]
	elseif nelec >= length(orbital_list)
		n = 0
		for i in orbital_list
			n = n | (1 << i)
		end
		result = [n]
	else
		restorb = orbital_list[1:end-1]
		thisorb = 1 << orbital_list[end]
		result = generate_string_iter(restorb, nelec)
		for n in generate_string_iter(restorb, nelec-1)
			tmp = (n | thisorb)
			push!(result, tmp)  
		end
	end
	return result
end

α_string = generate_string_iter(orbital_list,nα)
α_bitstring = [bitstring(i) for i in α_string]

display(α_string)
display(α_bitstring)

21-element Vector{Int64}:
  31
  47
  55
  59
  61
  62
  79
  87
  91
  93
   ⋮
 107
 109
 110
 115
 117
 118
 121
 122
 124

21-element Vector{String}:
 "0000000000000000000000000000000000000000000000000000000000011111"
 "0000000000000000000000000000000000000000000000000000000000101111"
 "0000000000000000000000000000000000000000000000000000000000110111"
 "0000000000000000000000000000000000000000000000000000000000111011"
 "0000000000000000000000000000000000000000000000000000000000111101"
 "0000000000000000000000000000000000000000000000000000000000111110"
 "0000000000000000000000000000000000000000000000000000000001001111"
 "0000000000000000000000000000000000000000000000000000000001010111"
 "0000000000000000000000000000000000000000000000000000000001011011"
 "0000000000000000000000000000000000000000000000000000000001011101"
 ⋮
 "0000000000000000000000000000000000000000000000000000000001101011"
 "0000000000000000000000000000000000000000000000000000000001101101"
 "0000000000000000000000000000000000000000000000000000000001101110"
 "0000000000000000000000000000000000000000000000000000000001110011"
 "0000000000000000

We want to solve the eigenvalue problem using an iterative method (Davidson algorithm). For this purpose we have to be able to calculate the matrix elements of the Hamiltonian efficiently. We need an efficient addressing scheme to map the monoexcited occupation strings and calculate only non-zero matrix elements of the Hamiltonian.

We will need two more functions before being able to generate a look up table containing all monoexcited occupation strings.

### Define the unique adress of a given excited string

In [7]:
function string2address(list_string, count, nmo, nel)
    address_map = Vector{Int64}(undef, count)
    nextaddr0 = binomial(nmo-1, nel)
    for i in 1:count
        string = list_string[i]
        addr = 1
        nelec_left = nel
        nextaddr = nextaddr0
        for norb_left=nmo-1:-1:0
            if ( (nelec_left == 0) || norb_left < nelec_left )
                break
            elseif (((1 << norb_left) & string) != 0)
            @assert nextaddr == binomial(norb_left,nelec_left)
            addr += nextaddr
            nextaddr *= nelec_left
            nextaddr /= norb_left
            nextaddr = floor(Int,nextaddr)
            nelec_left -= 1
            else
                nextaddr *= norb_left - nelec_left
                nextaddr /= norb_left
                nextaddr = floor(Int,nextaddr)
            end
        end
        address_map[i] = addr
    end
    return address_map
end

#List of all excited strings from ground state string
excited_string_int = [62, 94, 61, 93, 59, 91, 55, 87, 47, 79]
excited_string = [bitstring(i) for i in excited_string_int]
println("Ground state string α_string[1] = \n $(bitstring(α_string[1]))")
println("1st monoexcitation excited_string[1] = \n $(excited_string[1])")

stringaddress = string2address(excited_string_int,nα*nvir,nmo,nα)

println("Index of excited_string[1] in α_string is stringaddress[1] = $(stringaddress[1]) ")
println("α_string[6] = \n $(bitstring(α_string[6]))")


Ground state string α_string[1] = 
 0000000000000000000000000000000000000000000000000000000000011111
1st monoexcitation excited_string[1] = 
 0000000000000000000000000000000000000000000000000000000000111110


Index of excited_string[1] in α_string is stringaddress[1] = 6 
α_string[6] = 
 0000000000000000000000000000000000000000000000000000000000111110


### Determine the parity of the excited string
Due to anticommutation relations of creation and annihilation operators and the number of permutations required to put a given excited string in canonical order, we need to determine for each excited string a parity factor.

$${\{a_p^\dagger, a_q\}} = a_p^\dagger a_q+ a_q a_p^\dagger = \delta_{pq}$$
$${\{a_p^\dagger, a_q^\dagger\}} = 0$$
$${\{a_p, a_q\}} = 0$$

In [8]:
# Returns the parity of a given "a†(p) a(q)|string>"
function string_parity(p, q, string)
    if (p > q)
        mask = (1 << (p-1)) - (1 << q)
    else
        mask = (1 << (q-1)) - (1 << p)
    end
    if (count_ones(mask & string) % 2 == 1)
        parity = -1
    else 
        parity = 1
    end
    test = (string[1] & mask) 
    return parity
end

parity_62_string1 = string_parity(6, 2, α_string[1])

println("Ground state string α_string[1] = \n $(bitstring(α_string[1]))")
println("Parity for a†⁶a²   a¹†a²†a³†a⁴†a⁵†|vac> = $parity_62_string1" )


Ground state string α_string[1] = 
 0000000000000000000000000000000000000000000000000000000000011111
Parity for a†⁶a²   a¹†a²†a³†a⁴†a⁵†|vac> = -1


### Generate the monoexcitations $a_p^\dagger a_q \ket{det}$



In [9]:
function mono_string_index(strings, nmo, nocc)
    #Create look-up table of the effect of E_pq on a give string list
    # table = [a^†, a, address, parity]
    nvir = nmo - nocc
    allowed = nocc + nocc * nvir
    nstring = binomial(nmo,nocc)
    occupied = Vector{Int64}(undef, nocc)
    virtual = Vector{Int64}(undef, nvir)
    table = Array{Int64}(undef, allowed, 4, nstring)
    for (nstr, string) in enumerate(strings)
        o = 1
        v = 1
        #Mapping of creation and annihilation operators
        for i in 0:nmo-1
            if (string & (1 << i) != 0)
                occupied[o] = i + 1
                o += 1
            else 
                virtual[v] = i + 1
                v += 1
            end
        end

        for i in 1:nocc
            table[i,1,nstr] = occupied[i]
            table[i,2,nstr] = occupied[i]
            table[i,3,nstr] = nstr
            table[i,4,nstr] = 1
        end
        
        k = nocc + 1
        list_string = Vector{Int64}(undef,allowed-nocc)
        for i in 1:nocc
            for j in 1:nvir
                #Generate all excitations from a given "string" in "list_string"
                string_ex = (string ⊻ ( 1 << (occupied[i]-1)) | (1 << (virtual[j] - 1)))
                list_string[k - nocc] = string_ex
                table[k,1,nstr] = virtual[j]
                table[k,2,nstr] = occupied[i]
                table[k,4,nstr] = string_parity(virtual[j], occupied[i], string)
                k += 1
            end
            address = string2address(list_string,nocc*nvir,nmo,nocc)
            for k in 1:(nocc*nvir)
                table[k+nocc,3,nstr] = address[k]
            end
        end
    end
    return table
end

address_table = mono_string_index(α_string, nmo, nα)
println("α_bitstring[1] = $(α_bitstring[1])")
println("Corresponding entry - address_table[1]")
println("[a†, a, address, parity]")
address_table[:,:,1]

α_bitstring[1] = 0000000000000000000000000000000000000000000000000000000000011111
Corresponding entry - address_table[1]
[a†, a, address, parity]


15×4 Matrix{Int64}:
 1  1   1   1
 2  2   1   1
 3  3   1   1
 4  4   1   1
 5  5   1   1
 6  1   6   1
 7  1  11   1
 6  2   5  -1
 7  2  10  -1
 6  3   4   1
 7  3   9   1
 6  4   3  -1
 7  4   8  -1
 6  5   2   1
 7  5   7   1

Finally we make sure to store all orbital occupation in every possible string for later accessing the molecular orbitals

In [11]:
function occupied_strings(orblist, nel)    
    if (nel == 1)
        res = [[i] for i in orblist]
    elseif (nel >= length(orblist))
        res = [orblist] 
    else
        restorb = orblist[1:end-1]
        thisorb = orblist[end]
        res = occupied_strings(restorb, nel)
        for i in occupied_strings(restorb, nel-1)
            tmp = vcat(i, thisorb)
            push!(res, tmp)  
        end
    end
    return res
end

occupied_strings(orbital_list, nα)

21-element Vector{Vector{Int64}}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 5]
 [0, 1, 2, 4, 5]
 [0, 1, 3, 4, 5]
 [0, 2, 3, 4, 5]
 [1, 2, 3, 4, 5]
 [0, 1, 2, 3, 6]
 [0, 1, 2, 4, 6]
 [0, 1, 3, 4, 6]
 [0, 2, 3, 4, 6]
 ⋮
 [0, 1, 3, 5, 6]
 [0, 2, 3, 5, 6]
 [1, 2, 3, 5, 6]
 [0, 1, 4, 5, 6]
 [0, 2, 4, 5, 6]
 [1, 2, 4, 5, 6]
 [0, 3, 4, 5, 6]
 [1, 3, 4, 5, 6]
 [2, 3, 4, 5, 6]

And we put it all together...

In [12]:
function generate_index(orbital_list, nmo, nel)
    strings =  generate_string_iter(orbital_list,nel)
    address_table = mono_string_index(strings, nmo, nel)
    return address_table
end

generate_index (generic function with 1 method)

### The N-resolution method
Our goal is to solve for
$$\hat{H} = \sum_{pq}h_{pq} E_{pq} + \frac{1}{2} \sum_{pqrs}g_{pqrs} (E_{pq} E_{rs} - \delta_{pq} E_{ps} )$$

To solve iteratively the eigenvalue problem (using Davidson algorithm) we need to be able to define a vector contraction of the Hamiltonian $\sigma = HC$

The sigma vector in the alpha/beta string basis can be split and computed in two parts (one electron part $\sigma_{I_\alpha I_\beta}^{(1)}$ and a two electron part $\sigma_{I_\alpha I_\beta}^{(2)})$

$$ \sigma_{I_\alpha I_\beta} = \sigma_{I_\alpha I_\beta}^{(1)} +  \sigma_{I_\alpha I_\beta}^{(2)} $$


To simplify the expression of H, we first introduce effective one-electron integrals by absorbing the $g_{prrq}$  terms of the two-electron integral tensor.

$$k_{pq} = h_{pq} -\frac{1}{2} \sum_{r}^{N}g_{prrq} $$

$$ \sigma_{I_\alpha I_\beta}^{(1)} = \sum_{pq}\sum_{J_\alpha J_\beta}k_{pq} \bra{I_\alpha I_\beta}E_{pq}\ket{J_\alpha J_\beta} C_{J_\alpha J_\beta}$$

In [13]:
function absorb_h1e(hcore, C, eri, nmo, nel)
    h2e = copy(eri)
    h1e = C' * hcore * C
    @ein  jiik[j,k] := eri[j,i,i,k]
    f1e = h1e - jiik * 0.5
    f1e = f1e * (1. / (nel+1e-100))
    for k in 1:nmo
        h2e[:,:,k,k] .+= f1e
        h2e[k,k,:,:] .+= f1e
    end
    return 0.5 * h2e
end

absorb_h1e (generic function with 1 method)

Then we compute the two-electron part obtained by inserting the resolution of the identity.
$$ \sigma_{I_\alpha I_\beta}^{(2)} = \frac{1}{2} \sum_{K_\alpha K_\beta J_\alpha J_\beta} \sum_{pqrs} \bra{I_\alpha I_\beta}E_{pq}\ket{K_\alpha K_\beta} g_{pqrs} \bra{K_\alpha K_\beta}E_{rs}\ket{J_\alpha J_\beta} C_{J_\alpha J_\beta}$$

In [17]:
function contract2e(eri,ci0,nmo,nel)
    #RHF case only for now nα = nβ = nel // 2
    nstrings = binomial(nmo,nel)
    allowed = nel + (nmo - nel) * nel
    #Generate bitstring tables
    string_index_α = generate_index(collect(0:nmo-1), nmo, nel)
    string_index_β = generate_index(collect(0:nmo-1), nmo, nel)

    fcivec = reshape(ci0, (nstrings,nstrings))
    fcitensor = zeros(Float64, nstrings, nstrings, nmo, nmo)
    for i in axes(string_index_α, 3)
        str = view(string_index_α,:,:, i)
        for j in axes(str,1)
            c,a,adr,sgn = str[j,1],str[j,2],str[j,3],str[j,4]
            fcitensor[adr,:,a,c] += sgn * fcivec[i,:]
        end
    end

    for i in axes(string_index_β, 3)
        str = view(string_index_β,:,:, i)
        for j in axes(str,1)
            c,a,adr,sgn = str[j,1],str[j,2],str[j,3],str[j,4]
            fcitensor[:,adr,a,c] += sgn * fcivec[:,i]
        end    
    end
    @ein fcitensor[A,B,b,j] := eri[a,i,b,j] * fcitensor[A,B,a,i]
    cinew = zeros(nstrings, nstrings)

    for i in axes(string_index_α, 3)
        str = view(string_index_α,:,:, i)
        for j in axes(str,1)
            c,a,adr,sgn = str[j,1],str[j,2],str[j,3],str[j,4]
            cinew[adr,:] += sgn * fcitensor[i,:,a,c]
        end
    end
    for i in axes(string_index_β, 3)
        str = view(string_index_β,:,:, i)
        for j in axes(str,1)
            c,a,adr,sgn = str[j,1],str[j,2],str[j,3],str[j,4]
            cinew[:,adr] += sgn * fcitensor[:,i,a,c]
        end
    end
    return cinew
end



contract2e (generic function with 1 method)

For the Davidson algorithm, we also need to build a diagonal hamiltonian to use as a preconditionner

In [16]:
function make_hdiag(h1e, eri, nmo, nel)
    # α != β not handled
    occ_α = occupied_strings(collect(1:1:nmo),nel)
    occ_β = occupied_strings(collect(1:1:nmo),nel)
    jdiag = ein"iijj -> ij"(eri)
    kdiag = ein"ijji -> ij"(eri)
    hdiag = []
    for α in occ_α
        for β in occ_β
            e1 = sum([h1e[i,i] for i in α]) + sum([h1e[i,i] for i in β])
            e2 = sum(jdiag[α,:][:,α]) + sum(jdiag[α,:][:,β]) + sum(jdiag[β,:][:,α]) + sum(jdiag[β,:][:,β]) -sum(kdiag[α,:][:,α])- sum(kdiag[β,:][:,β])
            push!(hdiag, (e1 + (0.5 * e2))) 
        end
    end
    return hdiag
end

make_hdiag (generic function with 1 method)

###  Davidson algorithm


In [99]:
function davidson(sigma, ci0, preconditionner, nroots=1,silent = true, tol=1e-12, maxiter = 50, trial_space = 12)
    space = 0
    trial_space = trial_space + (nroots-1) * 3
    toloose = sqrt(tol)
    lindep = 1e-14
    nstrings = size(ci0)[1]
    dimt = nroots
    fresh_start = true
    v = Float64[]
    w = Float64[]
    e = Array{Float64}(undef, nroots)
    conv = [false for i in collect(1:1:nroots)]

    xt = Array{Float64}(undef,nstrings,nroots)
    axt = similar(xt)
    xs = Array{Float64}(undef,nstrings,nroots+trial_space)
    x0 = similar(xs)
    ax = similar(xs)
    ax0 = similar(xs)
    dx_norm = Array{Float64}(undef,nroots)

    for i in 1:dimt
        x0[:,i] = ci0[:,i]
    end
    
    heff = zeros((trial_space + nroots),(trial_space + nroots))
    norm_min = 1
    for it in 1:maxiter
        if fresh_start == true
            space = 1
            dimt = nroots
            xt = Array{Float64}(undef,nstrings,nroots)
            xs = Array{Float64}(undef,nstrings,nroots+trial_space)
            ax = similar(xs) 
            xt[:,1:dimt], dimt = orthonormalise(x0[:,1:dimt],dimt,lindep, silent)
            if dimt != nroots
                if silent == false
                    println("Warning - QR decomposition removed $(nroots - dimt) vectors ")
                end
            end
            x0 = similar(xs) 
        elseif space > 1
            xt[:,1:dimt], dimt  = orthonormalise(xt[:,1:dimt], dimt, lindep,silent)
        end

        for i in 1:dimt
            axt[:,i] = sigma(xt[:,i])
        end

        for (i,k) in enumerate(collect(space:1:(dimt+space-1)))
            xs[:,k] = xt[:,i]
            ax[:,k] = axt[:,i]
        end

        rnow = dimt
        head, space = space, space + rnow
        elast = copy(e)
        vlast = copy(v)
        conv_last = copy(conv)
        fill_heff!(heff,view(xs,:,1:space-1),view(ax,:,1:space-1), xt, axt, dimt)
        xt =  Array{Float64}(undef,nstrings,nroots)
        axt = similar(xt) 

        w, v = eigen(heff[1:space-1,1:space-1])
        
        e = w[1:nroots]
        v = v[:,1:nroots]

        x0 = Array{Float64}(undef,nstrings,nroots+trial_space)
        gen_x0(x0, v, view(xs,:,1:space-1),nstrings)
        gen_x0(ax0, v, view(ax,:,1:space-1),nstrings)
        
        elast, conv_last = sort_elast(elast, conv_last, vlast, v, fresh_start)
        de = e - elast
        dx_norm = Array{Float64}(undef,nroots)
        conv = [false for i in collect(1:1:nroots)]

        for (k, ek) in enumerate(e)
            xt[:,k] = ax0[:,k] - (ek * x0[:,k])
            dx_norm[k] = sqrt(dot(xt[:,k], xt[:,k]))
            conv[k] = (abs(de[k]) < tol) && (dx_norm[k] < toloose)
            if (conv[k] == true) && (conv_last[k] == false)
                if silent == false
                    println("root $k converged in $it iterations")
                end
            end
        end
        
        ax0 = Array{Float64}(undef,nstrings,nroots+trial_space)
        max_dx_norm = maximum(dx_norm)
        if all(conv)
            println("Davidson converged in $it iterations")
            break
        end

        if any(((!conv[k]) && (n^2 > lindep)) for (k,n) in enumerate(dx_norm))
            for (k, ek) in enumerate(e)
                if (!conv[k]) && (dx_norm[k]^2 > lindep)
                    xt[:,k] = preconditionner(xt[:,k], e[1])
                    xt[:,k] *= 1/sqrt(dot(xt[:,k], xt[:,k]))
                end
            end
        else
            for (k, ek) in enumerate(e)
                if dx_norm[k]^2 > lindep 
                    xt[:,k] = preconditionner(xt[:,k],e[1])
                    xt[:,k] *= 1 / sqrt(dot(xt[:,k], xt[:,k]))
                else
                    if silent == false
                        println("Remove the $k th eigenvector")
                    end
                end
            end
        end

        for i in 1:space-1
            for j in 1:dimt
                xt[:,j] -= xs[:,i] * dot(xs[:,i], xt[:,j])
            end
        end
        norm_min = 1

        for i in 1:dimt
            norm = sqrt(dot(xt[:,i], xt[:,i]))
            if norm^2 > lindep
                xt[:,i] *= 1 / norm
                norm_min = minimum([norm_min, norm])
            else
                if silent == false
                    println("Linear dependencies were found in the trial subspace")
                end
            end
        end

        if space == 1
            if silent == false
                println("Linear dependencies were found in the trial subspace")
            end
        end
    
        max_dx_last = max_dx_norm
        fresh_start = (space + nroots) > trial_space
        if it == maxiter
            println("WARNING ! - No convergence in $maxiter iterations")
        end
    end
    return e, x0[:,1:nroots]
end

function sort_elast(elast, conv_last, vlast, v, fresh_start)
    if fresh_start
        return elast, conv_last
    end
    head, nroots = size(vlast)
    ovlp = broadcast(abs, (v[1:head,:]' * vlast))
    idx = findmax.(eachrow(ovlp))
    new_order = [i[2] for i in idx]
    ordering_diff = (new_order != collect(1:1:length(idx)))
    if ordering_diff
        if silent == false
            println("Ordering of eigenstates changed : $ordering_diff ")
        end
    end
    return [elast[i[2]] for i in idx], [conv_last[i[2]] for i in idx]
end

function gen_x0(x0, v, xs, nstrings)
    space, nroots = size(v)
    x0[:,1:nroots] = ein"c,x -> xc"(v[space,:],xs[:,space])
    xsi = Array{Float64}(undef,nstrings, 1)
    for i in reverse(collect(1:1:(space-1)))
        xsi  = copy(xs[:,i])
        for k in 1:nroots
            x0[:,k] .+= v[i,k] * xsi
        end
        xsi = Array{Float64}(undef,nstrings, 1)
    end
end

function orthonormalise(xs, dimt, lindep=1e-14, silent=true)
    #QR decomposition to form a list of orthonormal vectors
    nstrings, nvec = size(xs)
    qs = zeros(nstrings,nvec)
    nv = 1
    for i in 1:nvec
        xi = copy(xs[:,i])
        for j in 1:nv
            prod = dot(qs[:,j], xi)
            xi -= prod * qs[:,j]
        end
        innerprod = dot(xi, xi)
        norm = sqrt(innerprod)
        if innerprod > lindep
            qs[:,nv] = xi / norm
            nv +=1
        else
            dimt -= 1
            if silent == false
                println("Warning QR decomposition removed vector n° $i")
            end
        end
    end
    return qs[:,1:(nv-1)], dimt 
end

function fill_heff!(heff,xs,ax,xt,axt,dimt)
    #Only for real Hamiltonian
    nrow = dimt
    row1 = size(ax)[2]
    row0 = row1 - nrow

    for (ip,i) in enumerate(range(row0, row1-1))
        for (jp,j) in enumerate(collect(row0:1:i-1))
            heff[i+1,j+1] = dot(xt[:,ip], axt[:,jp])
            heff[j+1,i+1] = heff[i+1,j+1]
        end
        heff[(i+1),(i+1)] = dot(xt[:,ip],axt[:,ip])
    end
    
    for i in 0:row0-1
        for (jp,j) in enumerate(range(row0,row1-1))
            heff[j+1,i+1] = dot(xt[:,jp], ax[:,i+1])
            heff[i+1,j+1] = heff[j+1,i+1]
        end
    end
end

fill_heff! (generic function with 1 method)

Now we are ready to run our first full-CI calculation

In [91]:
function FCI(C,nelec::Int64,ao_integrals,nroots,silent,tol, maxiter, trial_space)
    nmo = size(C)[1]
    nvir = nmo - (nelec÷2)
    nα = nelec÷2
    nβ = nα
    Hcore = Array(ao_integrals["T"]) + Array(ao_integrals["V"])
    μνλσ = Array(ao_integrals["ERI"])
    V_nuc = ao_integrals.molecule.Vnuc

    f1e = zeros(nmo,nmo)
    nstrings = binomial(nmo, nα)
    
    fcivec = zeros((nstrings * nstrings),nroots)
    j = 1
    for i in 1:nroots
        fcivec[j,i] = 1.0
        j += nstrings + i
    end
    
    #AO to MO transformation
    eri = ein"ai,bj,ck,dl,abcd->ijkl"(C,C,C,C,μνλσ)
    
    h1e = C' * Hcore * C
    h2e = absorb_h1e(Hcore, C, eri,nmo,nα*2)
    
    function σ_vec(v)
        σ = contract2e(h2e, v, nmo, nα)
        return vec(σ)
    end
    hdiag = make_hdiag(h1e,eri,nmo,nα)
    precond(x,e)= x ./ (hdiag .- e .+ 1e-4)
    
    e, c = davidson(σ_vec,fcivec,precond,nroots,silent, tol, maxiter, trial_space)

    for i in 1:nroots
        #println("E($i) = $(e[i]+V_nuc)")
        #display(c[:,i])
    end
    return e, c
end


FCI (generic function with 3 methods)

In [103]:
@molecule {
    O   0.000000     0.00000     0.11779
    H   0.000000     0.75545    -0.47116
    H   0.000000    -0.75545    -0.47116 
}
@set {
    basis sto-3g
    charge 0
    multiplicity 1 
}
ao_integrals = IntegralHelper(eri_type=Chonky())

#Run the Hartee-Fock Calculation
E, C = RHF(ao_integrals)
println("Converged Hartree-Fock Energy = $E")

e, c = FCI(C,10,ao_integrals,2,true,1e-12,50,4)
println("FCI Energy = $(e[1])")
println("FCI Energy = $(e[2])")


Iteration 1 ---- E = -118.16877249768544 ---- Ptol = 0.769869784390838
Iteration 2 ---- E = -71.19755398663607 ---- Ptol = 0.5276234409815913
Iteration 3 ---- E = -75.28186975157045 ---- Ptol = 0.053767482192072696
Iteration 4 ---- E = -74.9375074876783 ---- Ptol = 0.01050624624544192
Iteration 5 ---- E = -74.96647042201126 ---- Ptol = 0.004072569282002601
Iteration 6 ---- E = -74.96355897322383 ---- Ptol = 0.0016206824452923913
Iteration 7 ---- E = -74.96341516996763 ---- Ptol = 0.0006797065105307897
Iteration 8 ---- E = -74.96325755180108 ---- Ptol = 0.00028923338599482905
Iteration 9 ---- E = -74.96319518881691 ---- Ptol = 0.00012418820789279986
Iteration 10 ---- E = -74.96316753236361 ---- Ptol = 5.355248914878888e-5
Converged Hartree-Fock Energy = -74.96316753236361


Davidson converged in 30 iterations
FCI Energy = -84.2019975797477
FCI Energy = -83.60265109187954


In [106]:
#Helium atom
@molecule {
    He   0.12     0.01    0.156
}
@set {
    basis cc-pvtz
    charge 0
    multiplicity 1 
}

ao_integrals = IntegralHelper(eri_type=Chonky())

E, C = RHF(ao_integrals)
println("Converged Hartree-Fock Energy = $E")

e, c = FCI(C,2,ao_integrals,2,true,1e-12,50,8)
println("FCI Energy = $(e[1])")
println("FCI Energy = $(e[2])")


Iteration 1 ---- E = -3.997842064610413 ---- Ptol = 0.25269491357052354
Iteration 2 ---- E = -2.7562467972525058 ---- Ptol = 0.11972206831799641
Iteration 3 ---- E = -2.8508026651112046 ---- Ptol = 0.008229416928363584
Iteration 4 ---- E = -2.8598562449687885 ---- Ptol = 0.0012722635836544146
Iteration 5 ---- E = -2.86095917797117 ---- Ptol = 0.00023506696602524068
Iteration 6 ---- E = -2.8611214499323143 ---- Ptol = 4.1953621253439666e-5
Converged Hartree-Fock Energy = -2.8611214499323143


Davidson converged in 22 iterations
FCI Energy = -2.9002446160225204
FCI Energy = -1.7182397623113899
